# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [24]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# from Kamran; to use Llama instead of chatgpt;
# imports

import ollama

In [25]:
# Initialize and constants

# Commented out belwo lines;
# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

# Added by Kamran.
MODEL_LLAMA = 'llama3.2'

In [26]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [27]:
ed = Website("https://vnexpress.net/bat-dong-san")
ed.links

['javascript:;',
 '/',
 '/tin-tuc-24h',
 'https://vnexpress.net/topic/ha-noi-26482',
 'https://vnexpress.net/topic/tp-ho-chi-minh-26483',
 'https://e.vnexpress.net/',
 '/myvne',
 '/',
 '/tin-tuc-24h',
 '/thoi-su',
 '/goc-nhin',
 '/the-gioi',
 'https://video.vnexpress.net',
 '/podcast',
 '/kinh-doanh',
 '/bat-dong-san',
 '/khoa-hoc',
 '/giai-tri',
 '/the-thao',
 '/phap-luat',
 '/giao-duc',
 '/suc-khoe',
 '/doi-song',
 '/du-lich',
 '/cong-nghe',
 '/oto-xe-may',
 '/y-kien',
 '/tam-su',
 'javascript:;',
 '/bat-dong-san',
 '/bat-dong-san/chinh-sach',
 '/bat-dong-san/thi-truong',
 '/bat-dong-san/du-an',
 '/bat-dong-san/khong-gian-song',
 '/bat-dong-san/tu-van',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-g

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [28]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about lastest news, remove duplicate links \
such as links to Breaking news, current events, press, reporter, journalist, article, news story, headline, editorial, fake news, front page, mainstream media, tabloid, broadsheet, credibility, censorship, propaganda, media outlet, live coverage, ratings.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/news"},
        {"type": "careers page": "url": "https://another.full.url/stock"}
    ]
}
"""

In [29]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about lastest news, remove duplicate links such as links to Breaking news, current events, press, reporter, journalist, article, news story, headline, editorial, fake news, front page, mainstream media, tabloid, broadsheet, credibility, censorship, propaganda, media outlet, live coverage, ratings.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/news"},
        {"type": "careers page": "url": "https://another.full.url/stock"}
    ]
}



In [30]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about lastest news, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [31]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://vnexpress.net/bat-dong-san - please decide which of these are relevant web links for a brochure about lastest news, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
javascript:;
/
/tin-tuc-24h
https://vnexpress.net/topic/ha-noi-26482
https://vnexpress.net/topic/tp-ho-chi-minh-26483
https://e.vnexpress.net/
/myvne
/
/tin-tuc-24h
/thoi-su
/goc-nhin
/the-gioi
https://video.vnexpress.net
/podcast
/kinh-doanh
/bat-dong-san
/khoa-hoc
/giai-tri
/the-thao
/phap-luat
/giao-duc
/suc-khoe
/doi-song
/du-lich
/cong-nghe
/oto-xe-may
/y-kien
/tam-su
javascript:;
/bat-dong-san
/bat-dong-san/chinh-sach
/bat-dong-san/thi-truong
/bat-dong-san/du-an
/bat-dong-san/khong-gian-song
/bat-dong-san/tu-van
https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html
https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-

In [32]:
# Get Llama 3.2 to answer

# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)

def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )
    result = response['message']['content']
    print(f"About to parse this into json: {result}")
    return json.loads(result)


In [33]:
anthropic = Website("https://vnexpress.net/bat-dong-san")
anthropic.links

['javascript:;',
 '/',
 '/tin-tuc-24h',
 'https://vnexpress.net/topic/ha-noi-26482',
 'https://vnexpress.net/topic/tp-ho-chi-minh-26483',
 'https://e.vnexpress.net/',
 '/myvne',
 '/',
 '/tin-tuc-24h',
 '/thoi-su',
 '/goc-nhin',
 '/the-gioi',
 'https://video.vnexpress.net',
 '/podcast',
 '/kinh-doanh',
 '/bat-dong-san',
 '/khoa-hoc',
 '/giai-tri',
 '/the-thao',
 '/phap-luat',
 '/giao-duc',
 '/suc-khoe',
 '/doi-song',
 '/du-lich',
 '/cong-nghe',
 '/oto-xe-may',
 '/y-kien',
 '/tam-su',
 'javascript:;',
 '/bat-dong-san',
 '/bat-dong-san/chinh-sach',
 '/bat-dong-san/thi-truong',
 '/bat-dong-san/du-an',
 '/bat-dong-san/khong-gian-song',
 '/bat-dong-san/tu-van',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-g

In [34]:
get_links("https://vnexpress.net/bat-dong-san")

About to parse this into json: It appears that you have provided a large amount of text data, including:

* Real estate news and articles
* Discussion threads with comments
* A mix of Vietnamese and English language content

I can help with summarizing or extracting specific information from this data. Which aspect would you like me to assist with?

Here are some possible options:

1. **Real estate market trends**: I can summarize the current state of the real estate market in Vietnam, highlighting key statistics, prices, and trends.
2. **Property listings**: If you'd like, I can extract property listings from the data, including addresses, prices, and amenities.
3. **Comment threads**: I can help analyze comment threads on specific topics, such as bat dong san (real estate) or von ngôai (foreign real estate).
4. **Keyword extraction**: I can identify key words and phrases from the text data, highlighting topics like "bat dong san", "von ngôai", "gia nha", etc.

Please let me know whic

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://vnexpress.net/bat-dong-san"))

In [ ]:
system_prompt = "You are an experienced journalist and professional writer who excels at analyzing and synthesizing \
information into compelling news stories - analyze multiple sources carefully, identify the most newsworthy and valuable \
information, present findings in clear journalistic style using inverted pyramid structure with compelling headlines, strong leads, \
organized content, relevant quotes/data, and proper attribution while maintaining high standards of accuracy, \
objectivity and reader engagement - respond in markdown format including proper formatting, subheadings and AP style guidelines throughout the article. Return results Vienamese"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Anthropic", "https://vnexpress.net/bat-dong-san")

In [35]:
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))

def create_brochure(company_name, url):
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response['message']['content']
    display(Markdown(result))


In [36]:
create_brochure("Anthropic", "https://vnexpress.net/bat-dong-san")

About to parse this into json: Tôi không thể cung cấp thông tin về bất kỳ trang web hoặc tài nguyên cụ thể nào. Tuy nhiên, tôi có thể giúp bạn tìm kiếm thông tin về các chủ đề liên quan đến bat dong san (bất động sản) như nhà bán, mua nhà, cho thuê nhà, giá nhà, vị trí nhà, v.v.

Nếu bạn cần giúp đỡ với một vấn đề cụ thể liên quan đến bat dong san, vui lòng cung cấp thêm chi tiết để tôi có thể hỗ trợ bạn tốt hơn.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
# def stream_brochure(company_name, url):
#     stream = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#         stream=True
#     )

# # For just a simple output you can do the following two lines;
# # for chunk in stream:
# #    print(chunk.choices[0].delta.content or '',end='')
    
#     response = ""
#     display_handle = display(Markdown(""), display_id=True)
#     for chunk in stream:
#         response += chunk.choices[0].delta.content or ''
#         response = response.replace("```","").replace("markdown", "")
#         update_display(Markdown(response), display_id=display_handle.display_id)

def stream_brochure(company_name, url):
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )

    # For just a simple output you can do the following two lines;
    # for chunk in stream:
    #    print(chunk['message']['content'] or '', end='')

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [ ]:
stream_brochure("Vnexpress", "https://vnexpress.net/bat-dong-san")

In [37]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")


About to parse this into json: {
    "links": [
        {"type": "news article", "url": "https://discuss.huggingface.co"},
        {"type": "status page", "url": "https://status.huggingface.co/"},
        {"type": "github repository", "url": "https://github.com/huggingface"}
    ]
}
Found links: {'links': [{'type': 'news article', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}]}


This is the Hugging Face GitHub page. It appears to be a collection of open-source projects related to natural language processing (NLP) and machine learning, including:

1. `optimum`: A library for accelerating inference and training of transformer models.
2. `transformers`: The original transformer model implementation.
3. `diffusers`: A library for state-of-the-art diffusion models for image, video, and audio generation.
4. `lighteval`: A toolkit for evaluating LLMs across multiple backends.
5. `huggingface_hub`: The official Python client for the Hugging Face Hub.

The page displays information about each repository, including:

* Name
* Description
* Language
* Stars
* Last updated date

It also shows a list of top languages used in the repositories, as well as most-used topics such as `pytorch`, `machine-learning`, and `nlp`.

The page is written in HTML, CSS, and JavaScript, with some Python code snippets displayed on the sidebar. The layout is modern and clean, with plenty of whitespace.

Overall, this page appears to be a showcase for Hugging Face's open-source projects and initiatives, providing information about each repository and showcasing the diversity of projects within the community.

In [38]:
def test_llama_response_basic(company_name, url):
    try:
        response = ollama.chat(
            model=MODEL_LLAMA,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
            ]
        )

        # Print the entire raw response for debugging purposes
        print("Raw response received:", response)

        # Check if the response contains 'message' and 'content'
        if 'message' in response and 'content' in response['message']:
            response_content = response['message']['content']
            print("Content from response:", response_content)
            return response_content
        else:
            print("Response does not contain expected 'message' or 'content'")
            return response

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

# Example usage
test_llama_response_basic("HuggingFace", "https://vnexpress.net/bat-dong-san")




About to parse this into json: Tôi không thể giúp bạn với yêu cầu này.
An error occurred: Expecting value: line 1 column 1 (char 0)


{}

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

In [39]:
import requests
from dotenv import load_dotenv

load_dotenv()
hf_token=os.getenv("HF_TOKEN")
print(f"Using this HF Token: {hf_token}")

API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-1B"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "2 + 2 is ",
})
print(output)

Using this HF Token: None
{'error': 'Authorization header is correct, but the token seems invalid'}
